<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Soft-Voting-Edited-Dataset/blob/main/Cross_Project_SV_train_maven%2Btest_mina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
from numpy import mean
from numpy import std

In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [144]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [145]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [146]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-maven 3.3.3-3.3.9 - maven 3.3.3-3.3.9.csv")


In [147]:
weights['Change'] = weights['Change'].map({'yes': 1, 'no': 0})

In [148]:
weights.shape

(831, 8)

In [149]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,Change
0,22,10,26,237,1,73,13,0
1,2,0,18,40,0,40,5,0
2,4,0,24,71,0,27,11,0
3,2,2,16,15,0,0,3,0
4,3,0,14,7,0,0,1,0


In [150]:
X = weights.drop('Change', axis=1)
y = weights['Change']

In [151]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [152]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [153]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [154]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.7709090909090909, 0.7454545454545455, 0.6945454545454546]


In [155]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-mina data 2.0.7-2.0.8 - mina data 2.0.7-2.0.8.csv")

In [156]:
data['change'] = data['change'].map({'yes': 1, 'no': 0})

In [157]:
data.shape

(610, 8)

In [158]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,1,0,53,15,0,100,4,0
1,10,0,391,74,2,66,12,0
2,25,2,194,225,0,98,181,0
3,4,0,14,47,0,66,1,0
4,34,0,53,1049,0,0,40,0


In [159]:
X = data.drop('change', axis=1)
y = data['change']

In [160]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [161]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='soft', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [162]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 83.607


In [163]:
print(yhat)

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0]


In [164]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[98  2]
 [18  4]]


In [165]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = ensemble.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.7422727272727272


In [166]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.28769614180667996


In [167]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5809090909090909


In [168]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.42211588240886905


In [169]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.84      0.98      0.91       100
           1       0.67      0.18      0.29        22

    accuracy                           0.84       122
   macro avg       0.76      0.58      0.60       122
weighted avg       0.81      0.84      0.80       122

